# Introduction


As a part of the ***Magellan*** ecosystem, in this tutorial we will show how to use `deepmatcher` together with `Magellan`, to perform an end-to-end Entity Matching (EM) task. Specifically, `Magellan` performs EM in a **two-stage** fashion where given two tables,
1. Perform **blocking** on the two tables by removing obvious non-matching tuple pairs to get a candidate set K.
2. Perform **matching** to predict each pair in K as match or non-match. This stage consists of the following substeps:
    1. *Magellan* first helps the user take a sample S from K.
    2. The user labels all pairs in S as the training data.
    3. A classifer L will be learned on S.
    4. The user applies L to K to predict each pair as match or non-match.

Given the above workflow, `deepmatcher` fits in the step C, which is to take advantage of deep learning to learn a classifer. For the rest of the tutorial, we will use a real example, which is to match songs across two tables from iTunes and Amazon Music, to go through the workflow.

For more information on `Magellan`, please go to the project website: https://sites.google.com/site/anhaidgroup/projects/magellan.

# Step 0. Preparation
In order to use ***Magellan***, we need to first install it. The easiest way is to use "pip" as follow (please consult the package website for other installation options).

In [1]:
# Install Magellan.
try:
    import py_entitymatching as em
except:
    !pip install py_entitymatching

In [2]:
# Importdeepmatcher.
import deepmatcher as dm

import os

# Step 1. Load data
We first load the two input tables in the csv format using `Magellan`, that contain songs from iTunes and Amazon Music. These two tables are in the "example" directory included in our package.

In [3]:
# The path to the two input tables.
path_A = os.path.join('.', 'sample_data', 'itunes-amazon', 'tableA.csv')
path_B = os.path.join('.', 'sample_data', 'itunes-amazon', 'tableB.csv')

In [4]:
# Load the two tables.
A = em.read_csv_metadata(path_A, key='id')
B = em.read_csv_metadata(path_B, key='id')

In [5]:
# Basic information about the tables.
print('Number of tuples in A: ' + str(len(A)))
print('Number of tuples in B: ' + str(len(B)))
print('Number of tuples in A X B (i.e the cartesian product): ' + str(len(A)*len(B)))

Number of tuples in A: 6907
Number of tuples in B: 55923
Number of tuples in A X B (i.e the cartesian product): 386260161


In [6]:
# The first few tuples in table A.
A.head()

,id,Song_Name,Artist_Name,Album_Name,Genre,Price,CopyRight,Time,Released
0,0,Runaway Train,Cam,Welcome to Cam Country - EP,"Country , Music , Contemporary Country , Honky Tonk",$ 0.99,2015 Sony Music Entertainment,3:01,31-Mar-15
1,1,Track 14,Omi,Me 4 U,"Pop/Rock , Music , Pop , Dance , R&B / Soul",Album Only,"2015 Ultra Records , LLC under exclusive license to Columbia Records , a Division of Sony Music ...",3:41,NaN
2,2,Lips Are Movin,Meghan Trainor,Title,"Pop , Music , Pop/Rock , Dance , Rock , Teen Pop",$ 1.29,"2014 , 2015 Epic Records , a division of Sony Music Entertainment",3:01,9-Jan-15
3,3,I Want a Hippopotamus for Christmas,A Great Big World,I 'll Be Home For Christmas,"Holiday , Music",$ 1.29,"Compilation ( P ) 2014 Epic Records , a division of Sony Music Entertainment",2:20,24-Nov-14
4,4,Credit,Meghan Trainor,Title ( Deluxe ),"Pop , Music , Rock , Pop/Rock , Dance , Teen Pop",$ 1.29,"2014 , 2015 Epic Records , a division of Sony Music Entertainment",2:51,9-Jan-15


In [7]:
# The first few tuples in table B.
B.head()

,id,Song_Name,Artist_Name,Album_Name,Genre,Price,CopyRight,Time,Released
0,0,Saxophone Stomp [ Explicit ],Rusko,! ( Volume 2 ) [ Explicit ],"Dance & Electronic , Dubstep",$ 1.29,( C ) 2014 FMLY Under Exclusive License To Universal Music Canada Inc.,3:20,"September 16 , 2014"
1,1,I Wan na Mingle [ feat . Pusher ],Rusko,! ( Volume 2 ) [ Explicit ],"Dance & Electronic , Dubstep",$ 1.29,( C ) 2014 FMLY Under Exclusive License To Universal Music Canada Inc.,2:36,"September 16 , 2014"
2,2,Lytah,Rusko,! ( Volume 2 ) [ Explicit ],"Dance & Electronic , Dubstep",$ 1.29,( C ) 2014 FMLY Under Exclusive License To Universal Music Canada Inc.,3:48,"September 16 , 2014"
3,3,Slappy Pappy,Rusko,! ( Volume 2 ) [ Explicit ],"Dance & Electronic , Dubstep",$ 1.29,( C ) 2014 FMLY Under Exclusive License To Universal Music Canada Inc.,3:55,"September 16 , 2014"
4,4,Fushion Jam,Rusko,! ( Volume 2 ) [ Explicit ],"Dance & Electronic , Dubstep",$ 1.29,( C ) 2014 FMLY Under Exclusive License To Universal Music Canada Inc.,1:48,"September 16 , 2014"


# Step 2. Block tables to get the candidate set
We first perform blocking on A and B to get a candidate set K, by removing obvious non-matching tuple pairs. `Magellan` supports four different types of blocker: (1) attribute equivalence, (2) overlap, (3) rule-based, and (4) black-box. Typically, users need to mix and match these blockers with the debugging functionality provided in `Magellan` to get a good candidate set. (Developing a good blocker is not the focus of this tutorial. For more information on developing and debugging a blocker, please consult the user manual of `Magellan`.)

Here we show an example of blocking. Observe that matching tuple pairs should have some common words in the album name, so we first create an overlap blocker on the attribute name "Album_Name" with threshold 2, to remove all pairs with word overlap less 2 in that attribute.

In [8]:
# Create an overlap blocker in Magellan and apply it to A and B to get the candidate set K1 which is in the format of 
# a dataframe. The "l_out_attrs" and "r_out_attrs" parameters indicate the columns that will be included in K1 from A
# and B respectively.
ob = em.OverlapBlocker()
K1 = ob.block_tables(A, B, 'Album_Name', 'Album_Name',
                    l_output_attrs=['Song_Name', 'Artist_Name', 'Album_Name', 'Genre', 'Price', 'CopyRight', 'Time', 'Released'], 
                    r_output_attrs=['Song_Name', 'Artist_Name', 'Album_Name', 'Genre', 'Price', 'CopyRight', 'Time', 'Released'],
                    overlap_size=2)

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:18


In [9]:
# The number of tuple pairs in K1.
len(K1)

3152021

In [10]:
# The first few tuple pairs in K1
K1.head()

,_id,ltable_id,rtable_id,ltable_Song_Name,ltable_Artist_Name,ltable_Album_Name,ltable_Genre,ltable_Price,ltable_CopyRight,ltable_Time,ltable_Released,rtable_Song_Name,rtable_Artist_Name,rtable_Album_Name,rtable_Genre,rtable_Price,rtable_CopyRight,rtable_Time,rtable_Released
0,0,3995,0,Rudolph Blue,Brian Fechino,A Rock By The Sea Christmas : : Volume 2,"Holiday , Music , Rock",$ 0.99,2011 Hubbub !,1:57,8-Nov-11,Saxophone Stomp [ Explicit ],Rusko,! ( Volume 2 ) [ Explicit ],"Dance & Electronic , Dubstep",$ 1.29,( C ) 2014 FMLY Under Exclusive License To Universal Music Canada Inc.,3:20,"September 16 , 2014"
1,1,3995,1,Rudolph Blue,Brian Fechino,A Rock By The Sea Christmas : : Volume 2,"Holiday , Music , Rock",$ 0.99,2011 Hubbub !,1:57,8-Nov-11,I Wan na Mingle [ feat . Pusher ],Rusko,! ( Volume 2 ) [ Explicit ],"Dance & Electronic , Dubstep",$ 1.29,( C ) 2014 FMLY Under Exclusive License To Universal Music Canada Inc.,2:36,"September 16 , 2014"
2,2,3995,2,Rudolph Blue,Brian Fechino,A Rock By The Sea Christmas : : Volume 2,"Holiday , Music , Rock",$ 0.99,2011 Hubbub !,1:57,8-Nov-11,Lytah,Rusko,! ( Volume 2 ) [ Explicit ],"Dance & Electronic , Dubstep",$ 1.29,( C ) 2014 FMLY Under Exclusive License To Universal Music Canada Inc.,3:48,"September 16 , 2014"
3,3,3995,3,Rudolph Blue,Brian Fechino,A Rock By The Sea Christmas : : Volume 2,"Holiday , Music , Rock",$ 0.99,2011 Hubbub !,1:57,8-Nov-11,Slappy Pappy,Rusko,! ( Volume 2 ) [ Explicit ],"Dance & Electronic , Dubstep",$ 1.29,( C ) 2014 FMLY Under Exclusive License To Universal Music Canada Inc.,3:55,"September 16 , 2014"
4,4,3995,4,Rudolph Blue,Brian Fechino,A Rock By The Sea Christmas : : Volume 2,"Holiday , Music , Rock",$ 0.99,2011 Hubbub !,1:57,8-Nov-11,Fushion Jam,Rusko,! ( Volume 2 ) [ Explicit ],"Dance & Electronic , Dubstep",$ 1.29,( C ) 2014 FMLY Under Exclusive License To Universal Music Canada Inc.,1:48,"September 16 , 2014"


Notice that K1 has more than 3 million pair in it, which is too large to considered for matching. So we create an overlap blocking with threshold 1 on the attribute "Artist_Name" for K1, to filter all pairs in K1 that don't share any word in "Artist_Name". And we get the candidate set K2.

In [11]:
# Create a new overlap blocker to remove pairs from K1 that have no common word in "Artist_Name".
K2 = ob.block_candset(K1, 'Artist_Name', 'Artist_Name', overlap_size=1)

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:29


In [12]:
# The number of tuple pairs in K2.
len(K2)

167180

After the second blocking step, now we have a candidate set K2 with about 170K pairs. But we think it is still a bit larger to consider for matching. So we apply a third blocker, which is an overlap blocker on the attribute "Song_Name", to further reduce the size of the candidate set.

In [13]:
# Apply the third overlap blocker.
K3 = ob.block_candset(K2, 'Song_Name', 'Song_Name', overlap_size=1)

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:01


In [14]:
# The number of tuples pairs in K3.
len(K3)

38015

Now we have a candidate set with 38K pairs which is reasonable, so we take K3 as the final candidate set. We save the candidate to the disk in the csv format for future reuse.

In [15]:
path_K = os.path.join('.', 'sample_data', 'itunes-amazon', 'candidate.csv')
K3.to_csv(path_K, index=False)

# Step 3. Match tuple pairs in the candidate set
In this stage we will match tuple pairs in the candidate set to predict each of them as match or non-match. This is the part that `deepmatcher` will be involved. Specifically, it consists of the following steps:
1. Take a sample S from the candidate set K, and label all pairs in S.
2. Train a classifier L using S. Specifically, we will train a classifier using `deepmatcher`.
3. Apply L to the candidiate set K.

## Sample and label the candidate set
We first take a random sample S from the candidate set K using `Magellan`. Here for example, we sample 500 pairs for K. Then we label each pair as match (enter 1) or non-match (enter 0) and use S as the training data.

In [ ]:
# Take a sample of 500 pairs from the candidate set.
S = em.sample_table(K3, 500)

In [ ]:
# Label the sample S in a GUI. Enter 1 for match and 0 for non-match.
G = em.label_table(S, 'gold')

For the purposes of this tutorial, we will load in a pre-labeled dataset (of 539 tuple pairs) included in this package.

In [16]:
# The path to the labeled data file.
path_G = os.path.join('.', 'sample_data', 'itunes-amazon', 'gold.csv')

In [17]:
# Load the labeled data into a dataframe.
G = em.read_csv_metadata(path_G, 
                         key='_id',
                         ltable=A, rtable=B, 
                         fk_ltable='ltable_id', fk_rtable='rtable_id')
print('Number of labeled pairs:', len(G))

Number of labeled pairs: 539


## Train a classifier using labeled data
Once we have the labeled data, we use `deepmatcher` to train a classifier. The first thing we need to do is to split the data for training purpose. In this example, we split the labeled data into three parts: training, validation and test data, with the ratio of 3:1:1. (For now we only support spliting the labeled data into three parts train/valid/test, where the validation set is used for selecting the best model during the training epochs.) For the purpose of caching data and progressive training, we will first save the split parts to disk in the format of csv files, then load them back in. The cache file will be saved during the loading procedure. For subsequent training runs, the cache file will be used to save preprocessing time on the raw csv files, unless the csv files are modified (in this case, new cache file will be generated).

In [18]:
# The directory where the data splits will be saved.
split_path = os.path.join('.', 'sample_data', 'itunes-amazon')

In [19]:
# Split labeled data into train, valid, and test csv files to disk, with the split ratio of 3:1:1.
dm.data.split(G, split_path, 'train.csv', 'valid.csv', 'test.csv',
              [3, 1, 1])

In [20]:
# Load the training data files from the disk. Ignore the "left_id" and "right_id" 
# columns for data preprocessing.
train, validation, test = dm.data.process(
    path=os.path.join('.', 'sample_data', 'itunes-amazon'),
    cache='train_cache.pth',
    train='train.csv',
    validation='valid.csv',
    test='test.csv',
    left_prefix='ltable_',
    right_prefix='rtable_',
    id_attr='_id',
    ignore_columns=('ltable_id', 'rtable_id'))

Rebuilding data cache because: {'One or more data files have been modified.'}
Load time: 0.700446929782629
Vocab time: 13.431027501821518
Metadata time: 0.2899334393441677
Cache time: 0.5698287878185511


After we get the training data, we can use `deepmatcher` to train a classifier. Here we train a hybrid model.

In [21]:
# Create a hybrid model.
model = dm.MatchingModel(attr_summarizer='hybrid')

In [22]:
# Train the hybrid model with 10 training epochs, batch size of 16, positive-to-negative 
# ratio to be 3. We save the best model (with the 
# highest F1 score on the validation set) to 'hybrid_model.pth'.
model.run_train(
    train,
    validation,
    epochs=10,
    batch_size=16,
    best_save_path='hybrid_model.pth',
    pos_neg_ratio=3)

* Number of trainable parameters: 17757810
===>  TRAIN Epoch 1 :


0% [████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:04


Finished Epoch 1 || Run Time:    4.1 | Load Time:    0.5 || F1:  36.45 | Prec:  27.01 | Rec:  56.06 || Ex/s:  69.27

===>  EVAL Epoch 1 :


0% [█] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Finished Epoch 1 || Run Time:    0.6 | Load Time:    0.2 || F1:  49.06 | Prec:  65.00 | Rec:  39.39 || Ex/s: 141.96

* Best F1: 49.056603773584904
Saving best model...
===>  TRAIN Epoch 2 :


...

Finished Epoch 9 || Run Time:    0.6 | Load Time:    0.2 || F1:  81.69 | Prec:  76.32 | Rec:  87.88 || Ex/s: 134.07

===>  TRAIN Epoch 10 :


0% [████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:04


Finished Epoch 10 || Run Time:    3.8 | Load Time:    0.5 || F1:  99.25 | Prec:  98.51 | Rec: 100.00 || Ex/s:  74.70

===>  EVAL Epoch 10 :


0% [█] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Finished Epoch 10 || Run Time:    0.6 | Load Time:    0.2 || F1:  81.16 | Prec:  77.78 | Rec:  84.85 || Ex/s: 135.19

Loading best model...


Once we have the trained classifier, we can evaluate the accuracy using the test data.

In [23]:
# Evaluate the accuracy on the test data.
model.run_eval(test)

===>  EVAL Epoch 8 :
Finished Epoch 8 || Run Time:    0.3 | Load Time:    0.2 || F1:  88.24 | Prec:  85.71 | Rec:  90.91 || Ex/s: 200.00



88.23529411764706

## Apply the trained classifier to the candidate set
Now we have a trained classifer, we can apply it to the candidate set to predict each pair as match or non-match. To achieve this, we first load the trained model and the candidate set.

In [24]:
# Load the model.
model = dm.MatchingModel(attr_summarizer='hybrid')
model.load_state('hybrid_model.pth')

In [25]:
# Load the candidate set. Note that the trained model is an input parameter as we need to trained 
# model for candidate set preprocessing.
candidate = dm.data.process_unlabeled(
    path=os.path.join('.', 'sample_data', 'itunes-amazon', 'candidate.csv'),
    trained_model=model,
    ignore_columns=('ltable_id', 'rtable_id'))

Load time: 38.73918933980167
Vocab update time: 0.7370979888364673


After we load the candidate set, now we make the predictions using the `run_prediction` function. The *output_attribute* argument indicates the columns that will be included in the prediction table. Here 
we use all of the columns in the candidate set.

In [26]:
# Predict the pairs in the candidate set and return a dataframe containing the pair id with 
# the score of being a match.
predictions = model.run_prediction(candidate, output_attributes=list(candidate.get_raw_table().columns))

===>  PREDICT Epoch 8 :


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:02:26


Finished Epoch 8 || Run Time:   99.4 | Load Time:   47.8 || F1:   0.00 | Prec:   0.00 | Rec:   0.00 || Ex/s:   0.00



Below shows a few pairs in the prediction table. Note that this table is not sorted in the descending order of the 
match scores.

In [27]:
predictions.head()

,match_score,ltable_id,rtable_id,ltable_Song_Name,ltable_Artist_Name,ltable_Album_Name,ltable_Genre,ltable_Price,ltable_CopyRight,ltable_Time,ltable_Released,rtable_Song_Name,rtable_Artist_Name,rtable_Album_Name,rtable_Genre,rtable_Price,rtable_CopyRight,rtable_Time,rtable_Released
_id,,,,,,,,,,,,,,,,,,,
690110,0.994164,3354,17626,Extra Extra Credit,Wiz Khalifa,Flight School,"Hip-Hop/Rap , Music",$ 0.99,2009 Rostrum Records,4:03,17-Apr-09,Extra Extra Credit [ Explicit ],Wiz Khalifa,Flight School [ Explicit ],Rap & Hip-Hop,$ 0.99,2013 Mad Decent,4:03,"April 17 , 2009"
690109,0.994164,3353,17626,Extra Extra Credit,Wiz Khalifa,Flight School,"Hip-Hop/Rap , Music",$ 0.99,2009 Rostrum Records,4:03,17-Apr-09,Extra Extra Credit [ Explicit ],Wiz Khalifa,Flight School [ Explicit ],Rap & Hip-Hop,$ 0.99,2013 Mad Decent,4:03,"April 17 , 2009"
690108,0.994164,3336,17626,Extra Extra Credit,Wiz Khalifa,Flight School,"Hip-Hop/Rap , Music",$ 0.99,2009 Rostrum Records,4:03,17-Apr-09,Extra Extra Credit [ Explicit ],Wiz Khalifa,Flight School [ Explicit ],Rap & Hip-Hop,$ 0.99,2013 Mad Decent,4:03,"April 17 , 2009"
690209,0.806538,3354,17659,Extra Extra Credit,Wiz Khalifa,Flight School,"Hip-Hop/Rap , Music",$ 0.99,2009 Rostrum Records,4:03,17-Apr-09,Extra Credit [ Explicit ],Wiz Khalifa,Flight School [ Explicit ],Rap & Hip-Hop,$ 0.99,2013 Mad Decent,3:23,"April 17 , 2009"
690208,0.806538,3353,17659,Extra Extra Credit,Wiz Khalifa,Flight School,"Hip-Hop/Rap , Music",$ 0.99,2009 Rostrum Records,4:03,17-Apr-09,Extra Credit [ Explicit ],Wiz Khalifa,Flight School [ Explicit ],Rap & Hip-Hop,$ 0.99,2013 Mad Decent,3:23,"April 17 , 2009"


We can simply manipulate the prediction table above to get the pairs that we want. Suppose we are only interested in pairs with the match score more over 0.9, and we want to display them sorting in the descending order on match score. This can be achieved as follow.

In [28]:
high_score_pairs = predictions[predictions['match_score'] >= 0.9].sort_values(by=['match_score'], ascending=False)
high_score_pairs

,match_score,ltable_id,rtable_id,ltable_Song_Name,ltable_Artist_Name,ltable_Album_Name,ltable_Genre,ltable_Price,ltable_CopyRight,ltable_Time,ltable_Released,rtable_Song_Name,rtable_Artist_Name,rtable_Album_Name,rtable_Genre,rtable_Price,rtable_CopyRight,rtable_Time,rtable_Released
_id,,,,,,,,,,,,,,,,,,,
1274871,0.999357,4588,26140,Billy,Keith Urban,In the Ranch,"Country , Music , Honky Tonk , Contemporary Country",$ 1.29,1997 Capitol Nashville,3:46,1997,Billy,Keith Urban,Keith Urban In The Ranch,Country,$ 1.29,( c ) 1997 Capitol Nashville,3:46,"February 10 , 2004"
1274870,0.999357,4587,26140,Billy,Keith Urban,In the Ranch,"Country , Music , Honky Tonk , Contemporary Country",$ 1.29,1997 Capitol Nashville,3:46,1997,Billy,Keith Urban,Keith Urban In The Ranch,Country,$ 1.29,( c ) 1997 Capitol Nashville,3:46,"February 10 , 2004"
1274869,0.999357,4586,26140,Billy,Keith Urban,In the Ranch,"Country , Music , Honky Tonk , Contemporary Country",$ 1.29,1997 Capitol Nashville,3:46,1997,Billy,Keith Urban,Keith Urban In The Ranch,Country,$ 1.29,( c ) 1997 Capitol Nashville,3:46,"February 10 , 2004"
1274866,0.999357,4547,26140,Billy,Keith Urban,In the Ranch,"Country , Music , Honky Tonk , Contemporary Country",$ 1.29,1997 Capitol Nashville,3:46,1997,Billy,Keith Urban,Keith Urban In The Ranch,Country,$ 1.29,( c ) 1997 Capitol Nashville,3:46,"February 10 , 2004"
1274867,0.999357,4584,26140,Billy,Keith Urban,In the Ranch,"Country , Music , Honky Tonk , Contemporary Country",$ 1.29,1997 Capitol Nashville,3:46,1997,Billy,Keith Urban,Keith Urban In The Ranch,Country,$ 1.29,( c ) 1997 Capitol Nashville,3:46,"February 10 , 2004"
1274868,0.999357,4585,26140,Billy,Keith Urban,In the Ranch,"Country , Music , Honky Tonk , Contemporary Country",$ 1.29,1997 Capitol Nashville,3:46,1997,Billy,Keith Urban,Keith Urban In The Ranch,Country,$ 1.29,( c ) 1997 Capitol Nashville,3:46,"February 10 , 2004"
1571518,0.999352,433,33357,Nothing Would Be Better,Nick Jonas,Nick Jonas,"Pop , Music , Pop/Rock , Rock",$ 1.29,"2014 Island Records , a division of UMG Recordings , Inc. / Safehouse Records , LLC",4:34,10-Nov-14,Nothing Would Be Better,Nick Jonas,Nick Jonas [ Explicit ],Pop,$ 1.29,"( C ) 2014 Island Records/Safehouse Records LLC , a division of UMG Recordings , Inc.",4:34,"November 10 , 2014"
1572040,0.999352,394,33368,Nothing Would Be Better,Nick Jonas,Nick Jonas,"Pop , Music , Pop/Rock , Rock",$ 1.29,"2014 Island Records , a division of UMG Recordings , Inc. / Safehouse Records , LLC",4:34,10-Nov-14,Nothing Would Be Better,Nick Jonas,Nick Jonas [ Explicit ],Pop,$ 1.29,"( C ) 2014 Island Records/Safehouse Records LLC , a division of UMG Recordings , Inc.",4:34,"November 10 , 2014"
1571501,0.999352,407,33357,Nothing Would Be Better,Nick Jonas,Nick Jonas,"Pop , Music , Pop/Rock , Rock",$ 1.29,"2014 Island Records , a division of UMG Recordings , Inc. / Safehouse Records , LLC",4:34,10-Nov-14,Nothing Would Be Better,Nick Jonas,Nick Jonas [ Explicit ],Pop,$ 1.29,"( C ) 2014 Island Records/Safehouse Records LLC , a division of UMG Recordings , Inc.",4:34,"November 10 , 2014"
